In [ ]:
import sys
sys.path.append("../src")
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import gdal
import os
from deepgeo.dataset import rasterizer
import deepgeo.dataset.preprocessor as prep
import deepgeo.common.geofunctions as gf
import deepgeo.common.visualization as vis
import deepgeo.dataset.dataset_generator as dg
import deepgeo.dataset.data_augment as dtaug
import deepgeo.networks.model_builder as mb

In [ ]:
from pathlib import Path
#%% Load files
samples_path = Path('../data/chips/samples/')
labels_path = Path('../data/chips/labels/')
sample_files = [file for file in sorted(samples_path.glob('*.tif'))]
sample_imgs = []
label_imgs = []
for file in sample_files:    
    path_sample = file.resolve().as_posix()
    preproc = prep.Preprocessor(path_sample, no_data=-1)
    preproc.standardize_image('norm_range', {"min":0, "max":1})
    img_sample = preproc.get_array_stacked_raster()
    sample_imgs.append(img_sample)
    path_label = labels_path.joinpath(file.name).resolve().as_posix()
    img_label = gf.load_image(path_label,no_data=-1).astype(np.int32)
    label_imgs.append(img_label)


In [ ]:
indices = np.random.choice(len(sample_imgs), 4, replace=False)

import matplotlib.pyplot as plt
fig = plt.figure(figsize=(50, 50))

plt.subplot(4, 2, 1)
img = vis.plot_rgb_img(sample_imgs[indices[0]],
                       bands=[3,2,0],
                       contrast=True,
                       onlyreturn=True)
plt.imshow(img)
plt.subplot(4, 2, 2)
plt.imshow(label_imgs[indices[0]])

plt.subplot(4, 2, 3)
img = vis.plot_rgb_img(sample_imgs[indices[1]],
                       bands=[3,2,0],
                       contrast=True,
                       onlyreturn=True)
plt.imshow(img)
plt.subplot(4, 2, 4)
plt.imshow(label_imgs[indices[1]])

plt.subplot(4, 2, 5)
img = vis.plot_rgb_img(sample_imgs[indices[2]],
                       bands=[3,2,0],
                       contrast=True,
                       onlyreturn=True)
plt.imshow(img)
plt.subplot(4, 2, 6)
plt.imshow(label_imgs[indices[2]])

plt.subplot(4, 2, 7)
img = vis.plot_rgb_img(sample_imgs[indices[3]],
                       bands=[3,2,0],
                       contrast=True,
                       onlyreturn=True)
plt.imshow(img)
plt.subplot(4, 2, 8)
plt.imshow(label_imgs[indices[3]])

plt.show()


In [ ]:
dataset_description = {'years': 2018,
                       'range': {"min":0, "max":1},
                       'standardization': 'norm_range',
                       'indexes_to_compute': 'none',
                       'bands': ['b1', 'b2', 'b3', 'b4'],
                       'sensor': 'Worldview',
                       'classes': ['nodata','nopalm', 'palm'],
                       'img_no_data': -1,
                       'chip_size': 572,
                       'tolerance_nodata': .9,
                       'notes': 'Palm experiments.'}

In [ ]:
vis.plot_image_histogram(sample_imgs[indices[0]], cmap=["green", "red", "cyan", "blue"],
                         title="Normalized Raster", legend=dataset_description['bands'])

In [ ]:
output_ds = "../data/training2"
ds_file_name = "dataset"
model_dir = '../data/training2/model'

In [ ]:
train_tfrecord = os.path.join(output_ds, 'dataset_train.tfrecord')
test_tfrecord = os.path.join(output_ds, 'dataset_test.tfrecord')
val_dataset = os.path.join(output_ds, 'dataset_valid.npz')

In [ ]:
generator = dg.DatasetGenerator(sample_imgs,
                                label_imgs,
                                strategy='fileset',
                                description=dataset_description)
params = {'win_size': dataset_description['chip_size'], 'overlap': (0,0)}
generator.generate_chips(params)
chip_struct = generator.get_samples()

In [ ]:
generator.shuffle_ds()
generator.split_ds(perc_test=10, perc_val=10)

In [ ]:
chip_struct = generator.get_samples()
print(chip_struct.keys())

In [ ]:
generator.save_to_disk(output_ds, ds_file_name)

In [ ]:
params = {
    'network': 'unet',
    'epochs': 10000,
    'batch_size': 4,
    'chip_size': dataset_description['chip_size'],
    'bands': 4,
    'learning_rate': 0.01,
    'learning_rate_decay': True,
    'decay_rate': 0.999,
    'l2_reg_rate': 0.0005,
    'chips_tensorboard': 2,
    'loss_func': 'avg_soft_dice',
    'data_aug_ops': ['rot90', 'rot180', 'rot270', 'flip_left_right',
                     'flip_up_down', 'flip_transpose'],
    'data_aug_per_chip': 4,
    'num_classes': 3,
    'class_names': ['nodata', 'nopalm', 'palm'],
    'num_compositions': 1,
    'bands_plot': [[3, 2, 0]],
    'Notes': 'Palm experiment.'
}

In [ ]:
model = mb.ModelBuilder(params)

In [ ]:
%%capture output
model.train(train_tfrecord, test_tfrecord, model_dir)

In [ ]:
model = mb.ModelBuilder(model_dir)
dataset = np.load(val_dataset)
model.validate(dataset['chips'], dataset['labels'], model_dir, exclude_classes=[])

In [ ]:
# download = drive_auth.CreateFile({'id': '1AT-An70SGEGbgs9MOpWO3j1J39w6Fvv9'})
# download.GetContentFile('PDigital2018_22563_shp.zip')
# !unzip PDigital2018_22563_shp.zip

In [ ]:
 test_raster_file = '../data/cocais.tif'
 #test_shape_file = 'resources/2018/PDigital2018_22563__pol.shp'

In [ ]:
#out_labels_test = "resources/labels_test.tiff"

In [ ]:
preproc = prep.Preprocessor(test_raster_file, no_data=-1)
preproc.standardize_image('norm_range', {"min":0, "max":1})
raster_array = preproc.get_array_stacked_raster()

params_cg = {'overlap': [184, 184],
             'win_size': dataset_description['chip_size'],
             'raster_array': raster_array}

'''
generator = dg.DatasetGenerator(raster_array,
                                rasterized_layer,
                                strategy='sequential',
                                description=dataset_description)
generator.generate_chips(params_cg)
chip_struct = generator.get_samples()
'''
import deepgeo.dataset.sequential_chips as seqchips
chip_struct = seqchips.SequentialChipGenerator(params_cg).generate_chips()

In [ ]:
vis.plot_chips(chip_struct, raster_array, bands=[3, 2, 0], contrast=True)

In [ ]:
# trained_model = 'training_log'
trained_model = model_dir

model = mb.ModelBuilder(trained_model)
chips = model.predict(chip_struct, model_dir=trained_model)

In [ ]:
chips.keys()

In [ ]:
  output_pred = "../data/training2/probabilities.tif"
  gf.write_pred_chips(output_pred, test_raster_file, chips, chip_key='probabilities')

In [ ]:
pred_rarr = gdal.Open(output_pred).ReadAsArray()
vis.plot_labels(pred_rarr, params['class_names'], colors=['green', 'yellow'], title='Prediction')